In [ ]:
import numpy as np
import numpy.random as npr
import random
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

# More on Histograms for Data from Continuous Random Variables

Consider again observing values from a continuous random variable. Let's take two examples:

In [ ]:
E=stats.expon()
G=stats.norm()

In [ ]:
g=np.linspace(-5,5,101)
e=np.linspace(0,10,101)

In [ ]:
plt.plot(g,G.pdf(g))
plt.title("Density of a Gaussian(0,1) random variable");

In [ ]:
plt.plot(e,E.pdf(e))
plt.title("Density of an exponential(1) random variable");

Let's draw some observations from both random variables and try to understand how the histogram of the observations relates to the densities:

In [ ]:
Evals=E.rvs(size=100)
Gvals=G.rvs(size=100)

In [ ]:
plt.hist(Evals,bins=20)

In [ ]:
plt.hist(Gvals,bins=20)

We note from the graphs that the histograms have the same general shape as the densities for these random variables. 

Let's see why this is true: 

Let $X$ be a random variable with density $f_X(x)$. 

Now consider a bin $(x_1,x_2]$ over which the density $f_X(x)$ is continuous. Then there is $\tilde{x} \in (x_1, x_2]$ such that
\begin{align}
P \bigl( X \in  \left(x_1,x_2\right] \bigr) &= \int_{x_1}^{x_2} f_X(x)~dx \\
&= f_X(\tilde{x})(x_2-x_1)
\end{align}

Thus, the probability of a point lying in a bin is proportional to a density value in that bin and the width of the bin

We could turn the histogram into estimates of the bin probabilities by dividing all the entries by the total number of observations

Let $N$ denote the total number of observations. Let $B_i=(x_i, x_{i+1}]$ denote bin $i$, and $N_i$ denote the number of observations that fall in $B_i$, then 
$$
P \bigl( X \in  B_i) \approx \frac{N_i}{N}
$$

However, it makes more sense for our purposes to divide by a factor to get estimates of the density instead



Looking at the equation 
\begin{align}
P \bigl( X \in  \left(x_1,x_2\right] \bigr)
&= f_X(\tilde{x})(x_2-x_1),
\end{align}
we can approximate the density in bin $i$ by
$$
f_X(\tilde{x_i}) \approx \frac{N_i}{N \vert B_i \vert}
$$

Let's test. To make it easier to see, let's use bins with edges on the integers:

In [ ]:
mybins=range(7)
(counts,bins,_)=plt.hist(Evals,bins=mybins)

In [ ]:
plt.step(bins,np.append(counts,0)/np.sum(counts)/(bins[1]-bins[0])
         ,where='post')

Let's check if that is a valid density:

In [ ]:
np.sum(np.append(counts,0)/np.sum(counts)/(bins[1]-bins[0]))

Fortunately for us, we don't have to remember this formula or derive it whenever we want to create estimates of densities using a histogram from matplotlib.

Instead just call ```plt.hist``` with the keyword argument ```density=True```

In [ ]:
plt.hist(Evals, mybins, density=True);

In [ ]:
plt.hist(Gvals, density=True);

## Kernel Density Estimation

We have solved one of the problems in using histograms to approximate densities. 

But another problem remains: the blocky histogram does a poor job of approximating a continuous density

We can increase the number of bins to get a finer resolution, but then the number of observations in each bin get smaller, and the approximations for the density values may become poor:

In [ ]:
(counts,bins,_)=plt.hist(Gvals,bins=30, density=True);

Recall that we can build a histogram by putting a block for each observation, where the blocks are centered on the bins. 

We can use animation to visualize this better.  Because the animation is slow, let's use 40 random variables instead of 100:


In [ ]:
G40=G.rvs(size=40)

Here is an aminated GIF showing a histogram of the data being created step by step:

![](histogram.gif "Animated Histogram of Gaussian Data")

And here is the code to show the animation live in the browser

In [ ]:
from matplotlib import animation
#This will do an animated histogram
%matplotlib notebook

fig = plt.figure()
fig.set_dpi(100)
fig.set_size_inches(5, 4)
ymax=10

ax = plt.axes(xlim=(-3, 3), ylim=(0, ymax))
patches={}
floors={}

def init():
    patches['activePatch']=0
    for num,var in enumerate(G40):
        binedge=(var*2-0.5).round()/2
        patch = plt.Rectangle((binedge, ymax), 0.5,0.5, fc='b')
        ax.add_patch(patch)
        patches[num]=patch
        floors[binedge]=0.01
        #print(num,var, patches)


    return []

def animatePatch(i, patch, update):
    x, y = patch.xy
    #x = 2*patchnum + 3 * np.sin(np.radians(i))
    if update:
        y =  y-0.5
    patch.xy = (x, y)
    return patch,

def animate(i, patches):
    activePatch=patches['activePatch']
    x,y=patches[activePatch].xy
    if y<=floors[x]:
        floors[x]=floors[x]+0.5
        patches['activePatch']=activePatch+1
        activePatch+=1
        if activePatch not in patches.keys():
            return []
    #print(patches[activePatch])
    for patchnum in patches:
        #print(patchnum)
        if patchnum!='activePatch':
            animatePatch(i, patches[patchnum], activePatch==patchnum)
    
    return[]

anim = animation.FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=100*len(G40), 
                               fargs=(patches,),
                               interval=5,
                               blit=True,repeat=False)

#anim.save('histogram.gif', writer='imagemagick', fps=30)
plt.show()

What if we instead let each block be centered on the observation value?

Blocks don't stack neatly now, so we allow each part of the block to fall to the lowest open level.

Here is an aminated GIF showing this new type of histogram of the data being created step by step:

![](kde_rect.gif "Animated Histogram with Blocks Centered on Observations, Gaussian Data")

And here is the code to show the animation live in the browser



In [ ]:
# Now center blocks on the observed values
# Quantizing the space to make this work out
%matplotlib notebook


fig = plt.figure()
fig.set_dpi(100)
fig.set_size_inches(4, 3.5)

ax = plt.axes(xlim=(-5, 5), ylim=(0, 10))
blocks={}
floors={}
patchesPerBlock=9

def init():
    blocks['activeBlock']=0
    for num,var in enumerate(G40):
        center=round(var,1)
        blocks[num]=[]
        for i in range(patchesPerBlock):
            binedge=round(center+(i-patchesPerBlock//2)*0.1,1)
            patch = plt.Rectangle((binedge, 10), 0.1,0.5, fc='b')
            ax.add_patch(patch)
            blocks[num]+=[patch]
            floors[binedge]=0.01
        #print(num,var, patches)


    return []

def animatePatch(i, patch,update):
    return patch,

def animate(i, blocks):
    activeBlock=blocks['activeBlock']
    #print(activeBlock,blocks)
    numFloored=0
    for patch in blocks[activeBlock]:
        x,y=patch.xy
        if y<=floors[x]:
            #floors[x]=floors[x]+0.6
            #print("floors[",x,"]=",floors[x])
            numFloored+=1
        else:
            x,y=patch.xy
            patch.xy=(x,y-0.5)
    if numFloored>=patchesPerBlock:
        for patch in blocks[activeBlock]:
            x,y=patch.xy
            floors[x]=floors[x]+0.5
        blocks['activeBlock']=activeBlock+1
        activeBlock+=1
        if activeBlock not in blocks.keys():
            return []
        #print(patches[activePatch])
    for i in blocks:
        #print(patchnum)
        if i!='activeBlock':
            for patch in blocks[i]:
                animatePatch(i, patch,activeBlock==i)

    return[]

anim2 = animation.FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=100*len(G40), 
                               fargs=(blocks,),
                               interval=5,
                               blit=True,repeat=False)
anim2.save('kde_rect.gif', writer='imagemagick', fps=30)


plt.show()

This seems much closer to the true density. Here is a similar (non-animated) graph for 100 Gaussian RVs, with a finer quantization:

In [ ]:
%matplotlib inline
x = np.linspace(-5, 5, 1000)
width=2
density=np.zeros(x.size)
for xi in Gvals:
    block=(x>xi-width/2)&(x< xi+width/2)
    density+=block
plt.fill_between(x,density, color='b', alpha=0.1)

This is better, but not very smooth. There is no reason to only use rectangular blocks, so let's try a smoother shape.

In the animation below, I have replaced the rectangular block with a Gaussian shape.

This is even slower than the previous animations, so let's reduce down to 25 variables.

Here is an aminated GIF showing this new type of histogram of the data being created step by step:

![](kde_gauss.gif "Animated Histogram Built from Gaussian Shapes Centered on Observations, Gaussian Data")

And here is the code to show the animation live in the browser



In [ ]:
G25=stats.norm.rvs(size=25)

In [ ]:
# Trying to use non-rectangular shape
%matplotlib notebook


fig = plt.figure()
fig.set_dpi(100)
fig.set_size_inches(5, 4)

ymax=18
xmax=2.5
ax = plt.axes(xlim=(-xmax, xmax), ylim=(0, ymax))
blocks={}
floors={}
patchesPerBlock=25

def init():
    blocks['activeBlock']=0
    for num,var in enumerate(G25):
        center=round(var,1)
        blocks[num]=[]
        for i in range(patchesPerBlock):
            binedge=round(center+(i-patchesPerBlock//2)*0.1,1)
            patch = plt.Rectangle((binedge, ymax), 0.1,
                                  stats.norm.pdf((i-patchesPerBlock//2)*0.1,scale=0.5)*1.5, fc='b')
            ax.add_patch(patch)
            blocks[num]+=[patch]
            floors[binedge]=0.01
        #print(num,var, patches)


    return []

def animatePatch(i, patch,update):
    return patch,

def animate(i, blocks):
    activeBlock=blocks['activeBlock']
    if activeBlock not in blocks.keys():
            return []
    #print(activeBlock,blocks)
    numFloored=0
    for patch in blocks[activeBlock]:
        x,y=patch.xy
        if y<=floors[x]:
            numFloored+=1
        elif y-floors[x] >0.2:
            patch.xy=(x,y-0.2)
        elif y-floors[x] >0.1:
            patch.xy=(x,y-0.1)
        else:
            patch.xy=(x,y-0.05)
    if numFloored>=patchesPerBlock:
        for patch in blocks[activeBlock]:
            x,y=patch.xy
            #patch.xy=(x,0)
            new_height=y+patch.get_height()
            #new_height=floors[x]+patch.get_height()


            #patch.set_height(new_height)
            floors[x]=round(new_height,1)
        blocks['activeBlock']=activeBlock+1
        activeBlock+=1
        if activeBlock not in blocks.keys():
            return []
        #print(patches[activePatch])
    for i in blocks:
        #print(patchnum)
        if i!='activeBlock':
            for patch in blocks[i]:
                animatePatch(i, patch,activeBlock==i)

    return[]

anim3 = animation.FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=100*len(G25), 
                               fargs=(blocks,),
                               interval=5,
                               blit=True,repeat=False)
#anim
#anim3.save('kde_gauss.gif', writer='imagemagick', fps=60)


#plt.show()

Here is the same type of graph created with a wider, more finely quantized Gaussian shape and 100 Gaussian random variables:

In [ ]:
%matplotlib inline
x = np.linspace(-5,5, 1000)
density=np.zeros(x.size)
for xi in Gvals:
    ## Create new Gaussian RVs centered on the observation
    Gi=stats.norm(xi) 
    ## Use the density of the RV as the shape instead of a block:
    density+=Gi.pdf(x) 
    
plt.fill_between(x,density)

**Magic!** This looks a lot like our density, even though we only had 100 observations!

The shape we use to build this new density estimate is called a "kernel" 

This approach is called *kernel density estimation (KDE)*

However things are not quite as good as they seem:

In [ ]:
x = np.linspace(-5,5, 1000)
density=np.zeros(x.size)
for xi in Gvals:
    ## Create new Gaussian RVs centered on the observation
    Gi=stats.norm(xi) 
    ## Use the density of the RV as the shape instead of a block:
    density+=Gi.pdf(x) 
    
plt.fill_between(x,density/70) #Scale to make amplitudes match
plt.plot(x, G.pdf(x),'k')

Why? We only have 100 data points

In [ ]:
Gvals2=G.rvs(size=1000)
x = np.linspace(-5,5, 1000)
density=np.zeros(x.size)
for xi in Gvals2:
    ## Create new Gaussian RVs centered on the observation
    Gi=stats.norm(xi) 
    ## Use the density of the RV as the shape instead of a block:
    density+=Gi.pdf(x) 
    
plt.fill_between(x,density/700) #Scale to make amplitudes match
plt.plot(x, G.pdf(x),'k')

The esimated density is still wider than the true density. This is because the kernel we are using is too wide to capture the true density -- it provides too much smoothing. Let's try a narrower kernel:

In [ ]:
Gvals2=G.rvs(size=1000)
x = np.linspace(-5,5, 1000)
density=np.zeros(x.size)
for xi in Gvals2:
    ## Create new Gaussian RVs centered on the observation
    Gi=stats.norm(xi,scale=0.4) 
    ## Use the density of the RV as the shape instead of a block:
    density+=Gi.pdf(x) 
    
plt.fill_between(x,density/950) #Scale to make amplitudes match
plt.plot(x, G.pdf(x),'k')

That is pretty close. 

So one important issue in KDE is choosing the appropriate kernel.

The Gaussian kernel is most common, but the variance of the kernel must be chosen appropriately

Let's try some other variances:



In [ ]:
x = np.linspace(-5,5, 1000)
density=np.zeros(x.size)
for xi in Gvals2:
    ## Create new Gaussian RVs centered on the observation
    Gi=stats.norm(xi,scale=0.1) 
    ## Use the density of the RV as the shape instead of a block:
    density+=Gi.pdf(x) 
    
plt.fill_between(x,density)

In [ ]:
x = np.linspace(-5,5, 1000)
density=np.zeros(x.size)
for xi in Gvals2:
    ## Create new Gaussian RVs centered on the observation
    Gi=stats.norm(xi,scale=5) 
    ## Use the density of the RV as the shape instead of a block:
    density+=Gi.pdf(x) 
    
plt.fill_between(x,density)

Let's try an exponential kernel:

In [ ]:
x = np.linspace(-5,5, 1000)
density=np.zeros(x.size)
for xi in Gvals:
    ## Create new exponential RVs centered on the observation
    Ei=stats.expon(xi) 
    ## Use the density of the RV as the shape instead of a block:
    density+=Ei.pdf(x) 
    
plt.fill_between(x,density)

It is not that bad, but way worse than our Gaussian kernel.


Now consider applying KDE to the observations from the Exponential random variable:

In [ ]:
x = np.linspace(-3,8, 1000)
density=np.zeros(x.size)
for xi in Evals:
    Gi=stats.norm(xi)
    density+=Gi.pdf(x)
plt.fill_between(x,density/30)
plt.plot(x, E.pdf(x),'k')

Hmmm.... That doesn't look like an exponential density. Our random variable is non-negative!!!

We could do a little better if we make our shape more narrow:

In [ ]:
x = np.linspace(-3,8, 1000)
density=np.zeros(x.size)
for xi in Evals:
    Gi=stats.norm(xi,scale=0.3)
    density+=Gi.pdf(x)
plt.fill_between(x,density/60)
plt.plot(x, E.pdf(x),'k')

Finally, let's try our exponential kernel:

In [ ]:
x = np.linspace(0, 10, 1000)
density=np.zeros(x.size)
for xi in Evals:
    ## Create new exponential RVs centered on the observation
    Ei=stats.expon(xi) 
    ## Use the density of the RV as the shape instead of a block:
    density+=Ei.pdf(x) 
    
plt.fill_between(x,density/35)
plt.plot(x, E.pdf(x),'k')

Well, it is non-negative, but still not great

One, more time with a narrower kernel:

In [ ]:
x = np.linspace(0, 10, 1000)
density=np.zeros(x.size)
for xi in Evals:
    ## Create new exponential RVs centered on the observation
    Ei=stats.expon(xi,scale=0.3) 
    ## Use the density of the RV as the shape instead of a block:
    density+=Ei.pdf(x) 
    
plt.fill_between(x,density/70)
plt.plot(x, E.pdf(x),'k')

It is very hard to get a good estimate because the Exponential density is not continuous

In this class, we will apply KDE to continuous densities, and we will always use the Gaussian kernel.

However, we still need to choose the variance of the kernel.

Fortunately, there are techniques to automatically choose the variance of the Gaussian kernel. 

We will generally use scipy.stat's gaussian_kde method, which uses a heuristic to choose the variance of the Gaussian kernel (called the bandwidth)


In [ ]:
Gkde=stats.gaussian_kde(Gvals2)

In [ ]:
x=np.linspace(-5,5,1001)
plt.plot(x,Gkde(x))
plt.plot(x,G.pdf(x))